In [1]:
#import all necessary functions for working with ncdata
import xarray as xr
import matplotlib.pyplot as plt
from dask_jobqueue import PBSCluster
import seaborn as sns
import numpy as np

In [2]:
from utils import *

In [3]:
help(fix_time)

Help on function fix_time in module utils:

fix_time(ds)
    fix CESM monthly time-parsing bug



In [3]:
## read the data and stored in variable latin_hypercube
file='/glade/campaign/cgd/tss/projects/PPE/PPEn11_OAAT/CTL2010/hist/PPEn11_CTL2010_OAAT0000.clm2.h0.2005-02-01-00000.nc'
latin_hypercube = fix_time(xr.open_dataset(file))


In [1]:
#Dan's Recommendations
def yearly_weighted_average(da):
    # Get the array of number of days from the main dataset
    days_in_month = da['time.daysinmonth']
    group = variable_array.groupby("time.year")
    weighted_sum = ((da * days_in_month.loc[group.time.values]).sum(dim = 'time')) # Multiply each month's data by corresponding days in month
    total_days = days_in_month.loc[group.time.values].sum(dim = 'time')  # Total days in the year
    return weighted_sum / total_days  # Calculate weighted average for the year

In [8]:
# Step 1: Ask user for input
def get_user_input():
    # Ask the user for a variable name for which function calculates weighted average
    variable = input("Please enter the variable you are interested in: ")
    return variable
    
# Step 2: Store the user input
user_input = get_user_input()


# Step 3: Pass the user_input to the function that calculates average
def calculate_global_monthly_average(user_input):
    """Calculate the global monthly average based on the user input variable."""
    global latin_hypercube  # If dataset is a global variable and not passed as an argument
    # Changed any stored variable name to array
    variable_array = latin_hypercube[user_input]

    # Get the array of number of days from the main dataset
    days_in_month = variable_array['time.daysinmonth']

    # Group the data by year
    grouped_data = variable_array.groupby("time.year")

    # Define a function to calculate yearly weighted average
    #(days_per_month * dataArray).groupby('time.year').sum()
    
    def yearly_weighted_average(group):
        weighted_sum = ((group * days_in_month.loc[group.time.values]).sum(dim = 'time')) # Multiply each month's data by corresponding days in month
        total_days = days_in_month.loc[group.time.values].sum(dim = 'time')  # Total days in the year
        return weighted_sum / total_days  # Calculate weighted average for the year

    # Apply the yearly weighted average function to each group
    yearly_average = grouped_data.apply(yearly_weighted_average)

    return yearly_average

# Step 4: Call the function with user input and print the result
result = calculate_global_monthly_average(user_input)
print(result)


Please enter the variable you are interested in:  WIND


<xarray.DataArray (year: 10, gridcell: 400)>
array([[7.502123  , 7.26231056, 8.13464784, ..., 3.78583529, 5.58750981,
        5.04640036],
       [7.50257504, 7.26273746, 8.13461155, ..., 3.78586627, 5.73043097,
        5.04640081],
       [7.50257504, 7.26273746, 8.13461155, ..., 3.78586627, 6.5262402 ,
        5.04640081],
       ...,
       [4.86462182, 4.75366784, 9.30011424, ..., 3.71488366, 5.79567467,
        4.62294831],
       [4.98031804, 4.76255908, 9.66027202, ..., 3.44166691, 5.82658151,
        4.26770626],
       [4.80154061, 4.65924952, 9.12413873, ..., 3.44350884, 5.58235547,
        4.10407174]])
Coordinates:
  * year     (year) int64 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
Dimensions without coordinates: gridcell


In [ ]:
## test for a variable
calculate_global_monthly_average()

In [ ]:
#  ## testing line by line codes
# dataset = latin_hypercube
# variable  = dataset["TSA"]
# variable.groupby('time.month').mean('time')
# x = variable.groupby('time.year')

# days_in_month =  dataset['time.daysinmonth']
# dataset.groupby("time.year").apply(lambda x: (x * days_in_month).sum())
